# PoseEstimation_Utils

Importo tutte le librerie/moduli utili.

Il modulo sys permette di fare in modo di avere la sottocartella "models" (interna a "SuperGluePretrainedNetwork) allo stesso livello del codice, così da richiamare le funzioni al suo interno (superpoint, superglue, utils)

In [1]:
import sys
sys.path.append('./SuperGluePretrainedNetwork/models')
from utils import VideoStreamer, frame2tensor

from superpoint import SuperPoint

from superglue import SuperGlue
import torch

import cv2
import numpy as np

Definizione della funzione che serve alla fine per ottenere gli angoli di eulero (misurati in gradi) a partire dalla matrice di rotazione ottenuta da recoverPose().

La funzione è stata presa da https://learnopencv.com/rotation-matrix-to-euler-angles/ ed è relativa ad una rotazione X,Y,Z adattando l'analoga funzione matlab "rotm2euler.m" (dove la rotazione risulta essere invece Z,Y,X)

In [2]:
def RotMatToEulAng(R):
    sy = np.sqrt(R[0, 0] ** 2 + R[1, 0] ** 2)
    singular = sy < 1e-6

    if not singular:
        x = np.arctan2(R[2, 1], R[2, 2])  
        y = np.arctan2(-R[2, 0], sy)     
        z = np.arctan2(R[1, 0], R[0, 0])

    else:
        x = np.arctan2(-R[1, 2], R[1, 1])
        y = np.arctan2(-R[2, 0], sy)
        z = 0

    euler_angles_rad = np.array([x, y, z])
    euler_angles_deg = np.degrees(euler_angles_rad)

    return euler_angles_deg

Riga di debug per verificare che il path sia corretto: se restituisce "True" allora ha riconosciuto il percorso in modo corretto

In [3]:
# import os
# print(os.path.exists('C:/Users/antoa/ImageClassification/SuperGluePretrainedNetwork/MoonLu/MoonImg_Only2'))

Inizializzo l'oggetto con la classe VideoStreamer, dove do in input:
* la cartella con le sole due immagini da processare (dalla repo si capisce che le immagini possono essere di più, ma ho semplificato il codice per verificarne il corretto funzionamento)
* resize in un formato adatto delle immagini prese
* skip=2 permette di considerare tutte le immagini almeno una volta, senza saltarne nessuna
* image_glob per intendere il formato da prendere in considerazione

N.B. Per un giorno ho avuto il problema relativo ad un errore del tipo "tuple out of index", ed era relativo al fatto che Image_glob deve essere una lista con più eleemnti, per questo ho inserito .png in più formati. Inizialmente credevo fosse perchè avevo inserito male il path della cartella

le righe successive prendono le immagin iuna alla volta (incrementando il contatore ogni volta che next_frame viene chiamata) e la converte in un tensore

IMPORTANTE:

essendo il codice una versione iniziale che ha ancora dei problemi, se si vuole cambiare la coppia di immagini da analizzare bisogna farlo manualmente. Dalle immagini simulate della luna fornite da Luca, bisogna prenderne una a scelta tra quella relativa alla rotazione al roll,pitch o yaw e lasciarla nella cartella "MoonImg_Only2" insiema all'immagine di Nadir che è usata come riferimento.

In [4]:
images = VideoStreamer(basedir="./SuperGluePretrainedNetwork/MoonLu/MoonImg_Only2", resize=(640, 480), skip=2, image_glob=["*.png", "*.PNG"])

image0 = images.next_frame()
image1 = images.next_frame()

image0_tens = frame2tensor(image0[0], "cpu")
image1_tens = frame2tensor(image1[0], "cpu")

#print(type(image0_tens))
#print(image1)
#print(images.listing)

==> Processing image directory input: ./SuperGluePretrainedNetwork/MoonLu/MoonImg_Only2


Debug per assicurarsi di leggere tutte le immagini (se skip=1, prendeva tutte le immagini 2 volte ciascuna)

In [5]:
print(images.listing)

[WindowsPath('SuperGluePretrainedNetwork/MoonLu/MoonImg_Only2/Nadir.png'), WindowsPath('SuperGluePretrainedNetwork/MoonLu/MoonImg_Only2/roll_8_deg.png')]


Debug per vedere se il formato del tensore fosse corretto

In [6]:
# print(image0_tens)

Richiamo della funzione superpoint dalla cartella models, assegnandogli gli unici pesi disponibili e applicandole per le immagini tramite la funzione forward() che accetta un dizionario contente l'immagine e restituisce un altro dizionario in cui va a listare keypoint, gli score e le descrizioni di ognuno (quest'ultiam fatta in modo da riavere il matching con superglue se le descrizioni combaviano)

In [7]:
superpoint = SuperPoint ({'weights_path': 'superpoint_v1.pth'})

superimage0 = superpoint.forward({'image': image0_tens})
superimage1 = superpoint.forward({'image': image1_tens})

Loaded SuperPoint model


C:\Users\antoa\ImageClassification\SuperGluePretrainedNetwork/models\superpoint.py:137: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.load_state_dict(torch.load(str(pat

Debug per vedere il formato dell'output di forward()

In [8]:
# print(superimage0)

Applicazione di superglue in modo simile a superpoint.

Qui i pesi potevano essere selezionati tra indoor e outdoor, e la funzione forward accettava un dizionario contenente gli output di superpoint e le immagini interessate, ritornando i matches per le due immagini e la relativa affidabilità

In [9]:
superglue = SuperGlue({'weights_path': 'superglue_outdoor.pth'})

data = {
    'keypoints0': superimage0["keypoints"][0].unsqueeze(0),
    'keypoints1': superimage1["keypoints"][0].unsqueeze(0),
    'descriptors0': superimage0["descriptors"][0].unsqueeze(0),
    'descriptors1': superimage1["descriptors"][0].unsqueeze(0),
    'scores0': superimage0["scores"][0].unsqueeze(0),
    'scores1': superimage1["scores"][0].unsqueeze(0),
    'image0': torch.from_numpy(image0[0]).unsqueeze(0).unsqueeze(0),
    'image1': torch.from_numpy(image1[0]).unsqueeze(0).unsqueeze(0),
}

# Eseguire SuperGlue per trovare le corrispondenze
matches = superglue.forward(data)


C:\Users\antoa\ImageClassification\SuperGluePretrainedNetwork/models\superglue.py:226: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.load_state_dict(torch.load(str(path

Loaded SuperGlue model ("indoor" weights)


Debug per vedere il formato dell'output di forward()

In [10]:
# print(matches)

Prendiamo i keypoints delle due immagini e gli indici dei matches (a cui rimuoviamo una dimensione con squeeze() in vista del prossimo passaggio)

In [11]:
kpts0 = superimage0["keypoints"][0].numpy()
kpts1 = superimage1["keypoints"][0].numpy()

match0 = matches["matches0"].numpy().squeeze()
match1 = matches["matches1"].numpy().squeeze()

Alle prime due righe prendiamo i keypoint dalle due immagini eliminando i punti che non hanno avuto corrispondenze (quelli che avevano match -1)

Da riga 4 a 8 ho provato un metodo alternativo in merito alla creazione dei vettori keypoints da usare prima per la creazione della matrice essenziale e poi per la definizioned della posa:
assumendo che la variabile "match1", ottenuto dalla funzione superglue(), è un vettore che contiene gli indici del vettore kpts0 che corrispondono al vettore kpts1 (oppure -1 se non hanno match), ho provato ad usarlo per ordinare le coordinate nel vettore kpts0 eliminando tutti i  punti assenti di corrispondenze (così coem fatto alla riga 2 per keypoints1).

Imponendo la f e c secondo i parametri suggeriti da Luca, ho ricavato la matrice essenziale (facendo ottendere alla funzione stessa la Camera Matrix) e poi la matrice di rotazione e vettore traslazione con recoverPose(). Questi ultimi da capire meglio essendo che non mi è chiaro il sistema di riferimento e come osno definiti entrambi i parametri

IMPORTANTE I RISULTATI NELLA CELLA A SEGUIRE

In [12]:
# keypoints0 = kpts0[match0 != -1] #metodo 1
keypoints1 = kpts1[match1 != -1]

keypoints0 = np.empty((0,2), dtype=np.float32)               #metodo 2
for i in range(0,len(match1)):                               #metodo 2
    if match1[i] != -1:                                      #metodo 2
        index = match1[i]                                    #metodo 2
        keypoints0 = np.vstack([keypoints0, kpts0[index]])   #metodo 2

f = 6.013/(11.7e-3)
c = 1024/2
E, mask = cv2.findEssentialMat(keypoints0, keypoints1, focal=f, pp=(c, c), method=cv2.RANSAC)
_, R, t, mask_pose = cv2.recoverPose(E, keypoints0, keypoints1)

Stampa separata dei parametri per vederne i risutati

I vettori di keypoints0 e keypoints1 risultano avere la stessa lunghezza sia nel caso in cui li ordiniamo, sia nel caso in cui non li ordiniamo (fondamentale per far funzioanre findEssentialMar()), quindi primo check superato.
Controllando a capione le coordiante nei vari indici dei vettori, vediamo le corrispondenzeche ci servono, quindi secondo check superato.

IMPORTANTE:

si vede che la matrice essenziale E cambia effettivamente sia nel caso in cui non ordiniamo i vettori di keypoints, sia nel caso in cui li ordiniamo. Quindi questa è una variabile da tenere in considerazione.

In [13]:
# print(matches["matches1"])
# print(kpts0[493])
# print(kpts1[8])
# print(keypoints0[1])
# print(keypoints1[1])
print(E)

[[ 0.02217193 -0.52527643 -0.36333946]
 [ 0.49841155 -0.00074285  0.33922004]
 [ 0.36602448 -0.3053663   0.02949846]]


In [14]:
print(R)
print(t)

[[ 0.9973968  -0.00750699  0.07171655]
 [ 0.00823893  0.9999169  -0.00991567]
 [-0.07163616  0.01048073  0.99737576]]
[[-0.42796616]
 [-0.522523  ]
 [ 0.73743791]]


Applico la funzione di OpenCV per ottenere vettore rotazione a partire da matrice di rotazione (non ne ho capito molto, ma era una mossa disperata visto che non riesco a risolvere l'intoppo)

In [15]:
RotVect, _= cv2.Rodrigues(R)
print(np.degrees(RotVect))

[[0.58483143]
 [4.11039128]
 [0.45148705]]


Applico la funzione prima vista per ricavre gli angoli di eulero in gradi:
Semra essere vicino alla risoluzione corretta nel caso di una rotazione in yaw di 45° (manco troppo), ma non neglil altri casi

In [16]:
euler_angles = RotMatToEulAng(R)
print(euler_angles)

[0.6020592  4.107968   0.47327725]
